# Ray RLlib - Custom Environments and Reward Shaping

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

This lesson demonstrates how to adapt your own problem to use [Ray RLlib](http://rllib.io).

We cover two important concepts: 

1. How to create your own _Markov Decision Process_ abstraction.
2. How to shape the reward of your environment so make your agent more effective. 

In [ ]:
import numpy as np
import pandas as pd
import json, os, shutil, sys
import gym

import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

In [ ]:
sys.path.append('..') # so we can import from "util"
from util.line_plots import plot_line, plot_line_with_min_max, plot_line_with_stddev

In [ ]:
!../tools/start-ray.sh --check --verbose

In [ ]:
ray.init(address='auto', ignore_reinit_error=True)

In [ ]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

## Different Spaces

The first thing to do when formulating an RL problem is to specify the dimensions of your observation space and action space. Abstractions for these are provided in Gym. 

### Matching Different Actions to Their Corresponding Space

Let's familiarize ourselves with different Gym spaces. For example:

    discrete = spaces.Discrete(10)
    print("Random sample of this space: ", [discrete.sample() for i in range(4)])

Use `help(gym.spaces)` or `help([specific space])` (i.e., `help(gym.spaces.Discrete)`) for more info.

In [ ]:
help(gym.spaces)

In [ ]:
help(gym.spaces.Discrete)

Note the following example values in `action_space_examples` that the correspond to the declares spaces in `action_space_map`.

In [ ]:
from gym import spaces

action_space_map = {
    "discrete_10": spaces.Discrete(10),
    "box_1": spaces.Box(0, 1, shape=(1,), dtype=np.float64),  # the dtype can be omitted.
    "box_3x1": spaces.Box(-2, 2, shape=(3, 1), dtype=np.float64),
    "multi_discrete": spaces.MultiDiscrete([ 5, 2, 2, 4 ])
}

action_space_examples = {
    "discrete_10": 1,
    "box_1": np.array([0.89089584]),
    "box_3x1": np.array([[-1.2657754], [-1.6528835], [ 0.5982418]]),
    "multi_discrete": np.array([0, 0, 0, 2]),
}

for space_id, state in action_space_examples.items():
    assert action_space_map[space_id].contains(state), (f'Looks like {space_id} to {state} is matched incorrectly.')

Here's a space with 10 discrete values, 0 through 9, from which we sample and then update a counts map.

In [ ]:
counts = {key:0 for key in range(10)}
counts

for i in range(200):
    key = spaces.Discrete(10).sample()
    counts[key] = counts[key] + 1
counts

You can have more than one dimension of discrete (or continuous) values.

In [ ]:
md = spaces.MultiDiscrete([ 5, 2, 2, 4 ])
[md.sample() for _ in range(20)]

Note that the values for each dimension in the discrete space are inclusive, but zero-offset. For example, in the samples shown, the first integer returned in the array is 0-4, inclusive.

In [ ]:
box = spaces.Box(-2, 2, shape=(3,2), dtype=np.float64)
[box.sample() for _ in range(20)]

### Exercise 1: A Custom Environment with Rewards

Now we'll create an `n-Chain` environment, which represents moves along a linear chain of states, with two actions:

     (0) **forward**: move along the chain but returns no reward
     (1) **backward**: returns to the beginning and has a small reward

The end of the chain, however, provides a large reward, and by moving **forward** at the end of the chain, this large reward can be repeated.

#### Step 1: Implement `ChainEnv._setup_spaces`

Use a `spaces.Discrete` action space and observation space. Implement `ChainEnv._setup_spaces` in `ChainEnv` so that `self.action_space` and `self.obseration_space` are proper gym spaces.
  
1. The observation space is an integer in the range `[0 to n-1]`.
2. The action space is an integer in `[0, 1]`.

For example:

```python
self.action_space = spaces.Discrete(2)
self.observation_space = ...
```

You should see a message indicating tests passing when done correctly!

#### Step 2: Implement a reward function.

When `env.step` is called, it returns a tuple of `(state, reward, done, info)`. Right now, the reward is always 0. Modify `step()` so that the following rewards are returned for the given actions: 

1. `action == 1` will return `self.small_reward`.
2. `action == 0` will return 0 if `self.state < self.n - 1`.
3. `action == 0` will return `self.large_reward` if `self.state == self.n - 1`.

You should see a message indicating tests passing when done correctly. 

In [ ]:
from test_exercises import test_chain_env_spaces, test_chain_env_reward, test_chain_env_behavior
from gym import spaces

In [ ]:
class ChainEnv(gym.Env):
    
    def __init__(self, env_config = None):
        env_config = env_config or {}
        self.n = env_config.get("n", 20)
        self.small_reward = env_config.get("small", 2)  # payout for 'backwards' action
        self.large_reward = env_config.get("large", 10)  # payout at end of chain for 'forwards' action
        self.state = 0  # Start at beginning of the chain
        self._horizon = self.n
        self._counter = 0  # For terminating the episode
        self._setup_spaces()
    
    def _setup_spaces(self):
        ##############
        # TODO: Implement this so that it passes tests
        self.action_space = None
        self.observation_space = None
        ##############

    def step(self, action):
        assert self.action_space.contains(action)
        if action == 1:  # 'backwards': go back to the beginning, get small reward
            ##############
            # TODO 2: Implement this so that it passes tests
            reward = -1
            ##############
            self.state = 0
        elif self.state < self.n - 1:  # 'forwards': go up along the chain
            ##############
            # TODO 2: Implement this so that it passes tests
            reward = -1
            self.state += 1
        else:  # 'forwards': stay at the end of the chain, collect large reward
            ##############
            # TODO 2: Implement this so that it passes tests
            reward = -1
            ##############
        self._counter += 1
        done = self._counter >= self._horizon
        return self.state, reward, done, {}

    def reset(self):
        self.state = 0
        self._counter = 0
        return self.state
    
# Tests here:
test_chain_env_spaces(ChainEnv)
test_chain_env_reward(ChainEnv)

### Train a Policy on the Environment 

Now we'll train a policy on the environment and evaluate the policy. You'll see that despite an extremely high reward, the policy has barely explored the state space. 

In order to proceed, we'll import an implementation of the previous exercise, but you should actually comment-out the next cell once you complete the previous exercise!

In [ ]:
from chain_env import ChainEnv

In [ ]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["sgd_minibatch_size"] = 64
trainer_config["num_sgd_iter"] = 10

In [ ]:
trainer = PPOTrainer(trainer_config, ChainEnv);
for i in range(20):
    print(f'Training iteration {i:3d}...')
    trainer.train()

In [ ]:
env = ChainEnv({})
state = env.reset()

done = False
max_state = -1
cumulative_reward = 0

while not done:
    action = trainer.compute_action(state)
    state, reward, done, results = env.step(action)
    max_state = max(max_state, state)
    cumulative_reward += reward

print(f'Cumulative reward you've received is: {cumulative_reward}. Congratulations!')
print(f'Max state you visited is: {max_state}. This is out of {env.n} states.')

We only visited a small number of states, maybe only 1 (max == 0?).

## Shaping the Reward to Encourage Desired Behavior

We see that despite an extremely high reward, the policy has barely explored the state space. This is often the situation - where the reward designed to encourage a particular solution is suboptimal, and the behavior created is unintended.

### Exercise 2: Improve the Policy

Modify `ShapedChainEnv.step()` in the next cell to provide a reward that encourages the policy to traverse the chain (not just stick to 0). Do not change the behavior of the environment (the action -> state behavior should be the same).

You can change the reward to be whatever you wish. We'll text it in the next section.

In [ ]:
class ShapedChainEnv(ChainEnv):
    def step(self, action):
        assert self.action_space.contains(action)
        if action == 1:  # 'backwards': go back to the beginning
            reward = -1
            self.state = 0
        elif self.state < self.n - 1:  # 'forwards': go up along the chain
            reward = -1
            self.state += 1
        else:  # 'forwards': stay at the end of the chain
            reward = -1
        self._counter += 1
        done = self._counter >= self._horizon
        return self.state, reward, done, {}
    
test_chain_env_behavior(ShapedChainEnv)

### Evaluate `ShapedChainEnv` by Running the Cell(s) Below

This trains PPO on the new env and counts the number of states seen.

In [ ]:
trainer = PPOTrainer(trainer_config, ShapedChainEnv);
for i in range(20):
    print("Training iteration {}...".format(i))
    trainer.train()

env = ShapedChainEnv({})

max_states = []

for i in range(5):
    state = env.reset()
    done = False
    max_state = -1
    cumulative_reward = 0
    while not done:
        action = trainer.compute_action(state)
        state, reward, done, results = env.step(action)
        max_state = max(max_state, state)
        cumulative_reward += reward
    max_states += [max_state]

print(f'Cumulative reward you received is: {cumulative_reward}!')
print(f'Max states: {max_states}')
print(f'Max state you visited is: {max_state}. This is out of {env.n} states.')
assert (env.n - np.mean(max_states)) / env.n < 0.2, "This policy did not traverse many states."